In [1]:
#Libraries
import csv
import cv2
import numpy as np
import matplotlib.pyplot as plt 

from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda
from keras.layers import Convolution2D, Cropping2D
from keras.layers.pooling import MaxPooling2D

#VGG16
from keras.applications.vgg16 import VGG16

#resnet Model
from keras.applications.resnet50 import ResNet50

#GoogLeNet/Inception
from keras.applications.inception_v3 import InceptionV3

Using TensorFlow backend.


In [2]:
file_path = 'C:/Users/hhaggan/Desktop/BehaviorCloning/data/data/'

def GetData():
    lines = []
    images = []
    center = []
    left = []
    right = []
    measurements = []

    with open(file_path + 'driving_log.csv') as csvFile:
        reader = csv.reader(csvFile)
        next(reader, None)
        for line in reader:
            steering_center = float(line[3])

            # create adjusted steering measurements for the side camera images
            correction = 0.2 # this is a parameter to tune
            steering_left = steering_center + correction
            steering_right = steering_center - correction

            measurements.append(float(line[3]))
            center.append(file_path + '/' + line[0].strip())
            left.append(file_path + '/' + line[1].strip())
            right.append(file_path + '/' + line[2].strip())
    
    images.extend(center)
    images.extend(left)
    images.extend(right)
    
    return images, measurements

In [24]:
#data augmentation
def Augment_Data(images_path, measurements):
    augmented_images, augmented_measurements = [], []
    for image_path, measurement in zip(images_path, measurements):
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        augmented_images.append(cv2.flip(image, 1))
        augmented_measurements.append(measurement*-1.0)
    return augmented_images, augmented_measurements

'''X_train = np.array(images)
y_train = np.array(measurements)'''

'X_train = np.array(images)\ny_train = np.array(measurements)'

In [25]:
image_path, measurements = GetData()

In [26]:
images

['C:/Users/hhaggan/Desktop/BehaviorCloning/data/data//IMG/center_2016_12_01_13_30_48_287.jpg',
 'C:/Users/hhaggan/Desktop/BehaviorCloning/data/data//IMG/center_2016_12_01_13_30_48_404.jpg',
 'C:/Users/hhaggan/Desktop/BehaviorCloning/data/data//IMG/center_2016_12_01_13_31_12_937.jpg',
 'C:/Users/hhaggan/Desktop/BehaviorCloning/data/data//IMG/center_2016_12_01_13_31_13_037.jpg',
 'C:/Users/hhaggan/Desktop/BehaviorCloning/data/data//IMG/center_2016_12_01_13_31_13_177.jpg',
 'C:/Users/hhaggan/Desktop/BehaviorCloning/data/data//IMG/center_2016_12_01_13_31_13_279.jpg',
 'C:/Users/hhaggan/Desktop/BehaviorCloning/data/data//IMG/center_2016_12_01_13_31_13_381.jpg',
 'C:/Users/hhaggan/Desktop/BehaviorCloning/data/data//IMG/center_2016_12_01_13_31_13_482.jpg',
 'C:/Users/hhaggan/Desktop/BehaviorCloning/data/data//IMG/center_2016_12_01_13_31_13_584.jpg',
 'C:/Users/hhaggan/Desktop/BehaviorCloning/data/data//IMG/center_2016_12_01_13_31_13_686.jpg',
 'C:/Users/hhaggan/Desktop/BehaviorCloning/data/da

In [27]:
images, measurements = Augment_Data(image_path, measurements) 

In [10]:
measurements

[-0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0617599,
 -0.05219137,
 -0.05219137,
 -0.3679529,
 -0.5784606,
 -0.5784606,
 -0.1670138,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.08089697,
 -0.0904655,
 -0.0904655,
 -0.1574452,
 -0.1765823,
 -0.1765823,
 -0.0,
 -0.0,
 -0.0,
 0.0787459,
 0.0787459,
 0.0787459,
 0.0787459,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.05219137,
 -0.05219137,
 -0.0904655,
 -0.38709,
 -0.3583844,
 -0.05219137,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 0.03127411,
 0.05975719,
 0.05975719,
 0.05975719,
 0.05975719,
 0.05975719,
 0.05975719,
 0.05975719,
 0.05975719,
 0.05975719,
 0.05975719,
 0.04076847,
 -0.0,
 -0.0,
 -0.01391724,

In [28]:
images

[array([[[131, 163, 210],
         [131, 163, 210],
         [131, 163, 210],
         ...,
         [130, 163, 208],
         [130, 163, 208],
         [130, 163, 208]],
 
        [[131, 163, 210],
         [131, 163, 210],
         [131, 163, 210],
         ...,
         [130, 163, 208],
         [130, 163, 208],
         [130, 163, 208]],
 
        [[132, 164, 211],
         [132, 164, 211],
         [132, 164, 211],
         ...,
         [131, 164, 209],
         [131, 164, 209],
         [131, 164, 209]],
 
        ...,
 
        [[146, 129, 111],
         [142, 123, 108],
         [134, 117,  99],
         ...,
         [119, 106,  90],
         [117, 104,  88],
         [112,  99,  83]],
 
        [[129, 111,  91],
         [136, 115,  98],
         [142, 124, 104],
         ...,
         [114,  98,  82],
         [118, 105,  88],
         [122, 109,  92]],
 
        [[139, 118,  99],
         [146, 125, 106],
         [155, 134, 115],
         ...,
         [111,  95,  79],
  

In [31]:
X_train = np.array(images)
y_train = np.array(measurements)

model = Sequential()
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((70, 25), (0,0))))
model.add(Convolution2D(24,5,5, subsample=(2,2), activation='relu'))
#model.add(MaxPooling2D())
model.add(Convolution2D(36,5,5, subsample=(2,2),activation='relu'))
#model.add(MaxPooling2D())
model.add(Convolution2D(48,5,5, subsample=(2,2), activation='relu'))
model.add(Convolution2D(64,3,3,activation='relu'))
model.add(Convolution2D(64,3,3,activation='relu'))
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, epochs=5)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(24, (5, 5), activation="relu", strides=(2, 2))`
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(36, (5, 5), activation="relu", strides=(2, 2))`
  if __name__ == '__main__':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (5, 5), activation="relu", strides=(2, 2))`
  # This is added back by InteractiveShellApp.init_path()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  if sys.path[0] == '':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation

Train on 6428 samples, validate on 1608 samples
Epoch 1/5
6428/6428 [==============================] - 52s 8ms/step - loss: 0.0124 - val_loss: 0.0104
Epoch 2/5
6428/6428 [==============================] - 51s 8ms/step - loss: 0.0102 - val_loss: 0.0113
Epoch 3/5
6428/6428 [==============================] - 51s 8ms/step - loss: 0.0097 - val_loss: 0.0105
Epoch 4/5
6428/6428 [==============================] - 50s 8ms/step - loss: 0.0097 - val_loss: 0.0099
Epoch 5/5
6428/6428 [==============================] - 50s 8ms/step - loss: 0.0092 - val_loss: 0.0098
